# NFL Winner Predictions (Current Week)

This notebook pulls near-real-time NFL game data from ESPN's public scoreboard API, builds simple team ratings from season-to-date results, and predicts winners for the most current week of games. No API keys are required.

Approach:
- Fetch current week (season year, week, season type) from ESPN.
- Gather completed games from previous weeks this season to compute basic team ratings using points scored/allowed and net rating.
- Add a small home-field advantage.
- Convert rating differences to a win probability and pick winners.

Note: This is a lightweight baseline model using readily available data. It won’t match betting markets but updates quickly and runs anywhere.


In [1]:
# Dependencies
import sys
import json
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple

# Auto-install lightweight deps if missing
try:
    import requests  # type: ignore
    import pandas as pd  # type: ignore
except Exception:
    !{sys.executable} -m pip install --quiet requests pandas
    import requests  # type: ignore
    import pandas as pd  # type: ignore

ESPN_SCOREBOARD_URL = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
HOME_FIELD_ADVANTAGE_POINTS = 1.5  # simple constant
LOGISTIC_SCALE = 7.0  # larger => flatter probability curve

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)




[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def get_json(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    response = requests.get(url, params=params, timeout=20)
    response.raise_for_status()
    return response.json()


def get_current_week_info() -> Dict[str, Any]:
    data = get_json(ESPN_SCOREBOARD_URL)
    # Fallback handling if keys vary
    season = data.get("season", {})
    week = data.get("week", {})
    # ESPN uses seasonType: 1=Pre, 2=Reg, 3=Post
    info = {
        "season_year": season.get("year"),
        "season_type": season.get("type"),
        "week": week.get("number"),
        "raw": data,
    }
    return info


def fetch_week_scoreboard(season_year: int, season_type: int, week: int) -> Dict[str, Any]:
    params = {"year": season_year, "seasontype": season_type, "week": week}
    return get_json(ESPN_SCOREBOARD_URL, params=params)


def parse_games_from_scoreboard(board: Dict[str, Any]) -> pd.DataFrame:
    events: List[Dict[str, Any]] = board.get("events", [])
    rows: List[Dict[str, Any]] = []

    for ev in events:
        competitions = ev.get("competitions", [])
        if not competitions:
            continue
        comp = competitions[0]
        competitors = comp.get("competitors", [])
        if len(competitors) != 2:
            continue

        # Normalize home/away
        home = next((c for c in competitors if c.get("homeAway") == "home"), None)
        away = next((c for c in competitors if c.get("homeAway") == "away"), None)
        if home is None or away is None:
            continue

        def team_key(c: Dict[str, Any]) -> Tuple[str, str]:
            t = c.get("team", {})
            return t.get("id", ""), t.get("abbreviation", "")

        home_id, home_abbr = team_key(home)
        away_id, away_abbr = team_key(away)

        # Scores may be absent for future games
        try:
            home_score = float(home.get("score")) if home.get("score") is not None else None
        except Exception:
            home_score = None
        try:
            away_score = float(away.get("score")) if away.get("score") is not None else None
        except Exception:
            away_score = None

        status = ev.get("status", {}).get("type", {})
        status_desc = status.get("description")
        status_state = status.get("state")  # pre, in, post

        start_time_iso = ev.get("date")
        try:
            start_dt = datetime.fromisoformat(start_time_iso.replace("Z", "+00:00")) if start_time_iso else None
        except Exception:
            start_dt = None

        rows.append({
            "event_id": ev.get("id"),
            "season_year": board.get("season", {}).get("year"),
            "season_type": board.get("season", {}).get("type"),
            "week": board.get("week", {}).get("number"),
            "home_id": home_id,
            "home": home_abbr,
            "away_id": away_id,
            "away": away_abbr,
            "home_score": home_score,
            "away_score": away_score,
            "status_state": status_state,
            "status_desc": status_desc,
            "start_time": start_dt,
        })

    return pd.DataFrame(rows)


print("Fetchers ready.")


Fetchers ready.


In [3]:
def build_season_results_until_week(season_year: int, season_type: int, up_to_week_exclusive: int) -> pd.DataFrame:
    all_weeks: List[pd.DataFrame] = []
    for w in range(1, up_to_week_exclusive):
        try:
            board = fetch_week_scoreboard(season_year, season_type, w)
            df = parse_games_from_scoreboard(board)
            if not df.empty:
                # Keep only completed games for stats
                done = df[df["status_state"].isin(["post"])].copy()
                if not done.empty:
                    all_weeks.append(done)
        except Exception as e:
            print(f"Warning: failed to load week {w}: {e}")
            continue
    if not all_weeks:
        return pd.DataFrame(columns=[
            "team", "games", "points_for", "points_against", "ppg", "papg", "net_rating"
        ])

    games = pd.concat(all_weeks, ignore_index=True)

    # Build team-level aggregates
    home_stats = games[["home", "home_score", "away_score"]].rename(columns={
        "home": "team",
        "home_score": "points_for",
        "away_score": "points_against",
    })
    away_stats = games[["away", "away_score", "home_score"]].rename(columns={
        "away": "team",
        "away_score": "points_for",
        "home_score": "points_against",
    })

    tall = pd.concat([home_stats, away_stats], ignore_index=True)
    agg = tall.groupby("team", as_index=False).agg(
        games=("points_for", "count"),
        points_for=("points_for", "sum"),
        points_against=("points_against", "sum"),
    )
    agg["ppg"] = agg["points_for"] / agg["games"].clip(lower=1)
    agg["papg"] = agg["points_against"] / agg["games"].clip(lower=1)
    agg["net_rating"] = agg["ppg"] - agg["papg"]
    return agg


# Discover current week
current = get_current_week_info()
season_year = int(current["season_year"]) if current.get("season_year") else datetime.now(timezone.utc).year
season_type = int(current["season_type"]) if current.get("season_type") else 2  # default regular
current_week = int(current["week"]) if current.get("week") else 1

print({"season_year": season_year, "season_type": season_type, "current_week": current_week})

# Build ratings through prior week
ratings_df = build_season_results_until_week(season_year, season_type, max(1, current_week))
ratings_df.sort_values("net_rating", ascending=False, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)
ratings_df.head(10)


{'season_year': 2025, 'season_type': 2, 'current_week': 14}


,team,games,points_for,points_against,ppg,papg,net_rating
0,SEA,12,350.0,217.0,29.166667,18.083333,11.083333
1,LAR,12,334.0,210.0,27.833333,17.500000,10.333333
2,IND,12,357.0,249.0,29.750000,20.750000,9.000000
3,NE,13,351.0,241.0,27.000000,18.538462,8.461538
4,BUF,12,337.0,259.0,28.083333,21.583333,6.500000
5,DET,12,350.0,274.0,29.166667,22.833333,6.333333
6,KC,12,305.0,232.0,25.416667,19.333333,6.083333
7,GB,12,294.0,226.0,24.500000,18.833333,5.666667
8,DEN,12,284.0,218.0,23.666667,18.166667,5.500000
9,HOU,12,263.0,198.0,21.916667,16.500000,5.416667


In [4]:
import math

def win_prob_from_diff(rating_diff: float, scale: float = LOGISTIC_SCALE) -> float:
    # Logistic mapping from point differential to win probability
    return 1.0 / (1.0 + math.exp(-rating_diff / max(1e-6, scale)))


def predict_current_week_games(season_year: int, season_type: int, current_week: int, ratings: pd.DataFrame) -> pd.DataFrame:
    board = fetch_week_scoreboard(season_year, season_type, current_week)
    games = parse_games_from_scoreboard(board)
    if games.empty:
        return pd.DataFrame()

    # Index ratings by team abbr
    team_to_rating = ratings.set_index("team")["net_rating"].to_dict() if not ratings.empty else {}

    rows: List[Dict[str, Any]] = []
    for _, g in games.iterrows():
        home = g["home"]
        away = g["away"]
        home_rating = float(team_to_rating.get(home, 0.0))
        away_rating = float(team_to_rating.get(away, 0.0))

        # Home-field advantage added to home team rating
        diff_home_minus_away = (home_rating + HOME_FIELD_ADVANTAGE_POINTS) - away_rating
        prob_home = win_prob_from_diff(diff_home_minus_away)
        prob_away = 1.0 - prob_home

        predicted_winner = home if prob_home >= 0.5 else away
        confidence = max(prob_home, prob_away)

        rows.append({
            "season_year": int(g["season_year"]) if pd.notna(g["season_year"]) else season_year,
            "season_type": int(g["season_type"]) if pd.notna(g["season_type"]) else season_type,
            "week": int(g["week"]) if pd.notna(g["week"]) else current_week,
            "kickoff_utc": g["start_time"],
            "away": away,
            "home": home,
            "home_rating": home_rating,
            "away_rating": away_rating,
            "rating_diff_home_minus_away": diff_home_minus_away,
            "predicted_winner": predicted_winner,
            "win_prob_predicted": confidence,
            "status": g["status_state"],
            "status_desc": g["status_desc"],
        })

    pred = pd.DataFrame(rows)
    pred.sort_values(["win_prob_predicted", "kickoff_utc"], ascending=[False, True], inplace=True)
    pred.reset_index(drop=True, inplace=True)
    return pred

predictions_df = predict_current_week_games(season_year, season_type, current_week, ratings_df)
predictions_df


,season_year,season_type,week,kickoff_utc,away,home,home_rating,away_rating,rating_diff_home_minus_away,predicted_winner,win_prob_predicted,status,status_desc
0,2025,2,14,2025-12-07 18:00:00+00:00,CIN,BUF,6.500000,-7.833333,15.833333,BUF,0.905672,pre,Scheduled
1,2025,2,14,2025-12-07 21:05:00+00:00,DEN,LV,-10.750000,5.500000,-14.750000,DEN,0.891595,pre,Scheduled
2,2025,2,14,2025-12-07 18:00:00+00:00,SEA,ATL,-2.750000,11.083333,-12.333333,SEA,0.853448,pre,Scheduled
3,2025,2,14,2025-12-07 21:25:00+00:00,LAR,ARI,-3.166667,10.333333,-12.000000,LAR,0.847391,pre,Scheduled
4,2025,2,14,2025-12-07 18:00:00+00:00,NO,TB,-1.833333,-9.416667,9.083333,TB,0.785434,pre,Scheduled
5,2025,2,14,2025-12-07 18:00:00+00:00,TEN,CLE,-6.333333,-13.166667,8.333333,CLE,0.766826,pre,Scheduled
6,2025,2,14,2025-12-05 01:15:00+00:00,DAL,DET,6.333333,0.750000,7.083333,DET,0.733393,post,Final
7,2025,2,14,2025-12-07 21:25:00+00:00,CHI,GB,5.666667,0.500000,6.666667,GB,0.721594,pre,Scheduled
8,2025,2,14,2025-12-07 18:00:00+00:00,IND,JAX,3.250000,9.000000,-4.250000,IND,0.647289,pre,Scheduled
9,2025,2,14,2025-12-08 01:20:00+00:00,HOU,KC,6.083333,5.416667,2.166667,KC,0.576769,pre,Scheduled


In [6]:
# Save predictions to CSV (optional)
from pathlib import Path

now_str = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
out_dir = Path("predictions_output")
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = out_dir / f"nfl_predictions_w{current_week}_{season_year}_{now_str}.csv"
if predictions_df is not None and not predictions_df.empty:
    predictions_df.to_csv(csv_path, index=False)
    print(f"Saved predictions to: {csv_path.resolve()}\n")
else:
    print("No predictions generated (no current week games found).\n")

# Show a compact summary
cols = [
    "week", "kickoff_utc", "away", "home", "home_rating", "away_rating",
    "rating_diff_home_minus_away", "predicted_winner", "win_prob_predicted", "status_desc"
]
summary = predictions_df[cols].copy() if not predictions_df.empty else pd.DataFrame()
summary.head(50)


Saved predictions to: C:\Users\bmg32\OneDrive\Desktop\cs441FinalProj\predictions_output\nfl_predictions_w14_2025_20251205_184146.csv



,week,kickoff_utc,away,home,home_rating,away_rating,rating_diff_home_minus_away,predicted_winner,win_prob_predicted,status_desc
0,14,2025-12-07 18:00:00+00:00,CIN,BUF,6.500000,-7.833333,15.833333,BUF,0.905672,Scheduled
1,14,2025-12-07 21:05:00+00:00,DEN,LV,-10.750000,5.500000,-14.750000,DEN,0.891595,Scheduled
2,14,2025-12-07 18:00:00+00:00,SEA,ATL,-2.750000,11.083333,-12.333333,SEA,0.853448,Scheduled
3,14,2025-12-07 21:25:00+00:00,LAR,ARI,-3.166667,10.333333,-12.000000,LAR,0.847391,Scheduled
4,14,2025-12-07 18:00:00+00:00,NO,TB,-1.833333,-9.416667,9.083333,TB,0.785434,Scheduled
5,14,2025-12-07 18:00:00+00:00,TEN,CLE,-6.333333,-13.166667,8.333333,CLE,0.766826,Scheduled
6,14,2025-12-05 01:15:00+00:00,DAL,DET,6.333333,0.750000,7.083333,DET,0.733393,Final
7,14,2025-12-07 21:25:00+00:00,CHI,GB,5.666667,0.500000,6.666667,GB,0.721594,Scheduled
8,14,2025-12-07 18:00:00+00:00,IND,JAX,3.250000,9.000000,-4.250000,IND,0.647289,Scheduled
9,14,2025-12-08 01:20:00+00:00,HOU,KC,6.083333,5.416667,2.166667,KC,0.576769,Scheduled


In [7]:
# Plotting dependencies
import sys

try:
    import matplotlib.pyplot as plt  # type: ignore
    import seaborn as sns  # type: ignore
    import numpy as np  # type: ignore
except Exception:
    !{sys.executable} -m pip install --quiet matplotlib seaborn numpy
    import matplotlib.pyplot as plt  # type: ignore
    import seaborn as sns  # type: ignore
    import numpy as np  # type: ignore

sns.set(style="whitegrid")



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Matplotlib is building the font cache; this may take a moment.


In [8]:
def determine_actual_winner(row: pd.Series) -> Optional[str]:
    if pd.isna(row.get("home_score")) or pd.isna(row.get("away_score")):
        return None
    if row["home_score"] > row["away_score"]:
        return row["home"]
    if row["away_score"] > row["home_score"]:
        return row["away"]
    return None  # treat ties/unknown as None


def evaluate_week_predictions(season_year: int, season_type: int, week: int, ratings: pd.DataFrame) -> pd.DataFrame:
    board = fetch_week_scoreboard(season_year, season_type, week)
    games = parse_games_from_scoreboard(board)
    if games.empty:
        return pd.DataFrame()

    # Only completed games for evaluation
    games = games[games["status_state"].isin(["post"])].copy()
    if games.empty:
        return pd.DataFrame()

    team_to_rating = ratings.set_index("team")["net_rating"].to_dict() if not ratings.empty else {}

    rows: List[Dict[str, Any]] = []
    for _, g in games.iterrows():
        home = g["home"]
        away = g["away"]
        home_rating = float(team_to_rating.get(home, 0.0))
        away_rating = float(team_to_rating.get(away, 0.0))

        diff_home_minus_away = (home_rating + HOME_FIELD_ADVANTAGE_POINTS) - away_rating
        prob_home = win_prob_from_diff(diff_home_minus_away)
        prob_away = 1.0 - prob_home

        predicted_winner = home if prob_home >= 0.5 else away
        actual_winner = determine_actual_winner(g)
        if actual_winner is None:
            continue
        correct = predicted_winner == actual_winner
        y_true_home = 1 if actual_winner == home else 0
        prob_actual = prob_home if y_true_home == 1 else prob_away

        rows.append({
            "season_year": int(g["season_year"]) if pd.notna(g["season_year"]) else season_year,
            "season_type": int(g["season_type"]) if pd.notna(g["season_type"]) else season_type,
            "week": int(g["week"]) if pd.notna(g["week"]) else week,
            "kickoff_utc": g["start_time"],
            "away": away,
            "home": home,
            "home_score": g["home_score"],
            "away_score": g["away_score"],
            "home_rating": home_rating,
            "away_rating": away_rating,
            "prob_home": prob_home,
            "prob_away": prob_away,
            "predicted_winner": predicted_winner,
            "actual_winner": actual_winner,
            "correct": bool(correct),
            "y_true_home": y_true_home,
            "prob_actual": prob_actual,
        })

    return pd.DataFrame(rows)


def backtest_until_week(season_year: int, season_type: int, last_week_inclusive: int) -> pd.DataFrame:
    all_games: List[pd.DataFrame] = []
    for w in range(1, max(0, last_week_inclusive) + 1):
        ratings_prior = build_season_results_until_week(season_year, season_type, w)
        week_eval = evaluate_week_predictions(season_year, season_type, w, ratings_prior)
        if not week_eval.empty:
            all_games.append(week_eval)
    if not all_games:
        return pd.DataFrame(columns=[
            "season_year","season_type","week","kickoff_utc","away","home","home_score","away_score",
            "home_rating","away_rating","prob_home","prob_away","predicted_winner","actual_winner",
            "correct","y_true_home","prob_actual"
        ])
    return pd.concat(all_games, ignore_index=True)

print("Backtest helpers ready.")


Backtest helpers ready.


In [ ]:
# Backtest across past weeks and plot accuracy + calibration
last_week_inclusive = max(1, int(current_week))  # include current week; only completed games are used
backtest_df = backtest_until_week(season_year, season_type, last_week_inclusive)

if backtest_df.empty:
    print("No completed games available yet to evaluate.")
else:
    # Weekly and cumulative accuracy
    wk_counts = backtest_df.groupby("week", as_index=False).agg(
        correct_sum=("correct", "sum"),
        games=("correct", "count"),
        accuracy=("correct", "mean"),
    ).sort_values("week")
    wk_counts["cum_correct"] = wk_counts["correct_sum"].cumsum()
    wk_counts["cum_games"] = wk_counts["games"].cumsum()
    wk_counts["cum_accuracy"] = wk_counts["cum_correct"] / wk_counts["cum_games"]

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.lineplot(data=wk_counts, x="week", y="accuracy", marker="o", ax=axes[0])
    axes[0].set_title("Weekly accuracy")
    axes[0].set_ylim(0, 1)
    axes[0].set_xlabel("Week")
    axes[0].set_ylabel("Accuracy")

    sns.lineplot(data=wk_counts, x="week", y="cum_accuracy", marker="o", ax=axes[1], color="tab:green")
    axes[1].set_title("Cumulative accuracy")
    axes[1].set_ylim(0, 1)
    axes[1].set_xlabel("Week")
    axes[1].set_ylabel("Accuracy")
    plt.tight_layout()
    plt.show()

    # Calibration (reliability) plot using probability assigned to actual outcome
    backtest_df["prob_actual"] = np.where(backtest_df["y_true_home"] == 1, backtest_df["prob_home"], backtest_df["prob_away"])
    bins = np.linspace(0.0, 1.0, 11)
    backtest_df["prob_bin"] = pd.cut(backtest_df["prob_actual"], bins=bins, include_lowest=True)
    calib = backtest_df.groupby("prob_bin", as_index=False).agg(
        mean_pred=("prob_actual", "mean"),
        empirical=("correct", "mean"),
        n=("correct", "count"),
    ).dropna()

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.plot([0, 1], [0, 1], linestyle="--", color="gray", label="perfect")
    sns.scatterplot(data=calib, x="mean_pred", y="empirical", size="n", sizes=(20, 200), ax=ax)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xlabel("Predicted probability (actual outcome)")
    ax.set_ylabel("Observed frequency")
    ax.set_title("Calibration (reliability)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Brier score using home-win probability as the forecast
    y_true = backtest_df["y_true_home"].values.astype(float)
    brier = np.mean((backtest_df["prob_home"].values - y_true) ** 2)
    print(f"Brier score (home win prob): {brier:.3f}")

    # Display summary tables
    display(wk_counts)
    backtest_df.sort_values(["week", "kickoff_utc"]).head(20)
